<a href="https://colab.research.google.com/github/Saksham-tyagi1/NBA-Analysis/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
pd.set_option("display.max_colwidth", 1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd

def merge_csv_files(folder_path='/content/drive/MyDrive/NBA_Analysis', file_pattern='season_2023-24_chunk_'):
    """
    Merges multiple CSV files based on a file pattern and returns the merged DataFrame with optimized data types.
    """
    try:
        file_list = [f for f in os.listdir(folder_path) if f.startswith(file_pattern) and f.endswith('.csv')]
        if not file_list:
            print(f"No files matching the pattern '{file_pattern}' were found in the folder '{folder_path}'.")
            return None

        file_list.sort()
        print(f"Found {len(file_list)} files to merge: {file_list}")
        df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in file_list]
        final_df = pd.concat(df_list, ignore_index=True)

        final_df = optimize_data_types(final_df)

        print("Merging completed successfully.")
        return final_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def optimize_data_types(df):
    """
    Optimizes data types to reduce memory usage.
    """
    int_columns = df.select_dtypes(include=['int']).columns
    df[int_columns] = df[int_columns].apply(pd.to_numeric, downcast='integer')

    float_columns = df.select_dtypes(include=['float']).columns
    df[float_columns] = df[float_columns].apply(pd.to_numeric, downcast='float')

    object_columns = df.select_dtypes(include=['object']).columns
    for col in object_columns:
        if df[col].nunique() / len(df) < 0.5:
            df[col] = df[col].astype('category')

    return df

# Load and optimize df1
df1 = pd.read_csv('/content/drive/MyDrive/NBA_2024_Shots.csv')
df1 = optimize_data_types(df1)

# Load and merge extracted data
merged_df = merge_csv_files(folder_path="/content/drive/MyDrive/NBA_Analysis")

# Filter shot data
shot_data = merged_df[(merged_df['EVENTMSGTYPE'] == 1) | (merged_df['EVENTMSGTYPE'] == 2) |
                      (merged_df['EVENTMSGTYPE'] == 1.0) | (merged_df['EVENTMSGTYPE'] == 2.0)]

# Define relevant columns for shot analysis
relevant_columns = [
    'GAME_ID', 'PLAYER_ID', 'SPD', 'DIST', 'ORBC', 'DRBC', 'RBC',
    'TCHS', 'SAST', 'PASS', 'AST', 'FGM', 'FGA', 'FG_PCT_y',
    'DFGM', 'DFGA', 'DFG_PCT', 'SHOT_CLOCK_RANGE', 'SCORE', 'SCOREMARGIN',
    'EFG_PCT', 'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE'
]

filtered_extracted_df = shot_data[relevant_columns]
filtered_extracted_df = optimize_data_types(filtered_extracted_df)

# Ensure merge keys have consistent types
df1['GAME_ID'] = df1['GAME_ID'].astype(str)
df1['PLAYER_ID'] = df1['PLAYER_ID'].astype(int)
filtered_extracted_df['GAME_ID'] = filtered_extracted_df['GAME_ID'].astype(str)
filtered_extracted_df['PLAYER_ID'] = filtered_extracted_df['PLAYER_ID'].astype(int)

# Merge in chunks using left join and save to CSV
chunk_size = 50000
output_file = "complete_merged_shot_data.csv"

with open(output_file, 'w') as f:
    df1_chunk = df1.iloc[:chunk_size]
    merged_chunk = pd.merge(df1_chunk, filtered_extracted_df, on=['GAME_ID', 'PLAYER_ID'], how='left')
    merged_chunk = optimize_data_types(merged_chunk)
    merged_chunk = merged_chunk.drop_duplicates()

    merged_chunk.to_csv(f, index=False)

for start in range(chunk_size, len(df1), chunk_size):
    df1_chunk = df1.iloc[start:start + chunk_size]
    merged_chunk = pd.merge(df1_chunk, filtered_extracted_df, on=['GAME_ID', 'PLAYER_ID'], how='left')
    merged_chunk = optimize_data_types(merged_chunk)
    merged_chunk = merged_chunk.drop_duplicates()

    merged_chunk.to_csv(output_file, mode='a', index=False, header=False)

print(f"Completed merging. Full dataset saved to {output_file}")

# Load final merged data for verification
chunks = pd.read_csv('/content/complete_merged_shot_data.csv', chunksize=chunk_size)
df_list = []
for chunk in chunks:
    processed_chunk = optimize_data_types(chunk)
    df_list.append(processed_chunk)

df = pd.concat(df_list, ignore_index=True)

# Verify LOC_X and LOC_Y in the final DataFrame
print("Sample LOC_X values in final merged DataFrame:", df['LOC_X'].head())
print("Sample LOC_Y values in final merged DataFrame:", df['LOC_Y'].head())
print("Final DataFrame loaded with optimized data types.")


Found 12 files to merge: ['season_2023-24_chunk_0.csv', 'season_2023-24_chunk_1.csv', 'season_2023-24_chunk_10.csv', 'season_2023-24_chunk_11.csv', 'season_2023-24_chunk_2.csv', 'season_2023-24_chunk_3.csv', 'season_2023-24_chunk_4.csv', 'season_2023-24_chunk_5.csv', 'season_2023-24_chunk_6.csv', 'season_2023-24_chunk_7.csv', 'season_2023-24_chunk_8.csv', 'season_2023-24_chunk_9.csv']


<ipython-input-3-65a1eeaaf6ad>:16: DtypeWarning: Columns (6,7,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in file_list]
<ipython-input-3-65a1eeaaf6ad>:16: DtypeWarning: Columns (6,7,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in file_list]
<ipython-input-3-65a1eeaaf6ad>:16: DtypeWarning: Columns (6,7,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in file_list]
<ipython-input-3-65a1eeaaf6ad>:16: DtypeWarning: Columns (6,7,33,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in file_list]
<ipython-input-3-65a1eeaaf6ad>:16: DtypeWarning: Columns (6,7,58) have mixed types. Specify dtype option on impor

Merging completed successfully.


<ipython-input-3-65a1eeaaf6ad>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[int_columns] = df[int_columns].apply(pd.to_numeric, downcast='integer')
<ipython-input-3-65a1eeaaf6ad>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[float_columns] = df[float_columns].apply(pd.to_numeric, downcast='float')
<ipython-input-3-65a1eeaaf6ad>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

Completed merging. Full dataset saved to complete_merged_shot_data.csv


<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunks:
<ipython-input-3-65a1eeaaf6ad>:98: DtypeWarnin

Sample LOC_X values in final merged DataFrame: 0   -0.4
1    1.5
2   -3.3
3   -1.0
4   -0.0
Name: LOC_X, dtype: float32
Sample LOC_Y values in final merged DataFrame: 0    17.450001
1    30.549999
2    6.550000 
3    5.850000 
4    6.250000 
Name: LOC_Y, dtype: float32
Final DataFrame loaded with optimized data types.


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/NBA_2024_Shots.csv')

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/optimized_data.parquet')

In [ ]:
df.isnull().sum()

,0
SEASON_1,0
SEASON_2,0
TEAM_ID,0
TEAM_NAME,0
PLAYER_ID,0
PLAYER_NAME,0
POSITION_GROUP,0
POSITION,0
GAME_DATE,0
GAME_ID,0


In [ ]:
df_original = df.copy()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
pd.set_option("display.max_colwidth", 1)


In [ ]:

df = df.dropna(subset=['SCORE', 'SCOREMARGIN', 'POSITION_GROUP', 'POSITION']).reset_index(drop=True)


In [ ]:
# Convert to more memory-efficient types
df['PLAYER_ID'] = df['PLAYER_ID'].astype('int32')
df['GAME_ID'] = df['GAME_ID'].astype('int32')
df['EVENT_TYPE'] = df['EVENT_TYPE'].astype('category')
df['LOC_X'] = df['LOC_X'].astype('float32')
df['LOC_Y'] = df['LOC_Y'].astype('float32')
df['SHOT_DISTANCE'] = df['SHOT_DISTANCE'].astype('float32')
df['QUARTER'] = df['QUARTER'].astype('int8')
df['SECS_LEFT'] = df['SECS_LEFT'].astype('int8')
df['SPD'] = df['SPD'].astype('float32')
df['DIST'] = df['DIST'].astype('float32')
df['EFG_PCT'] = df['EFG_PCT'].astype('float32')



In [ ]:
pip install dask


In [ ]:
import dask.dataframe as dd

# Load data as a Dask DataFrame, splitting into more partitions if needed
ddf = dd.from_pandas(df, npartitions=20)

# Convert columns to optimized data types but avoid float16 for indexing columns
ddf['PLAYER_ID'] = ddf['PLAYER_ID'].astype('uint16')
ddf['GAME_ID'] = ddf['GAME_ID'].astype('uint32')
ddf['EVENT_TYPE'] = ddf['EVENT_TYPE'].astype('category')
ddf['LOC_X'] = ddf['LOC_X'].astype('float32')
ddf['LOC_Y'] = ddf['LOC_Y'].astype('float32')
ddf['SHOT_DISTANCE'] = ddf['SHOT_DISTANCE'].astype('float32')
ddf['QUARTER'] = ddf['QUARTER'].astype('uint8')
ddf['SECS_LEFT'] = ddf['SECS_LEFT'].astype('uint8')
ddf['SPD'] = ddf['SPD'].astype('float32')
ddf['DIST'] = ddf['DIST'].astype('float32')
ddf['EFG_PCT'] = ddf['EFG_PCT'].astype('float32')

# Specify the expected output structure for meta
meta = {
    'PLAYER_ID': 'uint16',
    'GAME_ID': 'uint32',
    'EVENT_TYPE': 'category',
    'LOC_X': 'float32',
    'LOC_Y': 'float32',
    'SHOT_DISTANCE': 'float32',
    'QUARTER': 'uint8',
    'shot_count': 'int32',
    'avg_secs_left': 'float32',
    'avg_spd': 'float32',
    'avg_dist': 'float32',
    'avg_efg_pct': 'float32'
}

# Groupby with the provided meta structure to avoid memory inference issues
result = (
    ddf.groupby(['PLAYER_ID', 'GAME_ID', 'EVENT_TYPE', 'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER'])
    .agg({
        'SECS_LEFT': 'mean',  # Average of seconds left
        'SPD': 'mean',        # Average speed
        'DIST': 'mean',       # Average distance
        'EFG_PCT': 'mean',    # Average effective FG percentage
        'EVENT_TYPE': 'size'  # Count of shots/events
    })
    .rename(columns={'EVENT_TYPE': 'shot_count'})
    .reset_index()
)

# Trigger computation and sort by shot count
final_agg_df = result.compute(meta=meta)
final_agg_df = final_agg_df.sort_values(by='shot_count', ascending=False)

# View the summary
print(final_agg_df.head())



/usr/local/lib/python3.10/dist-packages/dask/dataframe/groupby.py:1424: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


In [ ]:
df = df_original.copy()

filtered_df = df[df[['SCORE', 'SCOREMARGIN', 'POSITION_GROUP', 'POSITION']].notna().all(axis=1)]

unchanged_columns = ['LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'PLAYER_NAME']

for col in unchanged_columns:
    if not filtered_df[col].equals(df.loc[filtered_df.index, col]):
        print(f"Warning: Column '{col}' changed unexpectedly after filtering.")
    else:
        print(f"Column '{col}' is unchanged after filtering.")


Column 'LOC_X' is unchanged after filtering.
Column 'LOC_Y' is unchanged after filtering.
Column 'SHOT_DISTANCE' is unchanged after filtering.
Column 'PLAYER_NAME' is unchanged after filtering.


In [ ]:
numerical_columns = ['SPD', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'PASS',
                     'AST', 'FGM', 'FGA', 'FG_PCT_y', 'DFGM', 'DFGA', 'DFG_PCT',
                     'EFG_PCT', 'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE']

for col in numerical_columns:
    df[col] = df.groupby(['PLAYER_ID', 'GAME_ID'])[col].transform(lambda x: x.fillna(x.median()))




Data Cleaning


In [ ]:
print(sample_data['SCORE'].unique())


NameError: name 'sample_data' is not defined

In [ ]:
df['SCORE'].isnull().sum()

0

In [ ]:
df[['HOME_SCORE', 'AWAY_SCORE']] = df['SCORE'].str.split(' - ', expand=True)

df['HOME_SCORE'] = pd.to_numeric(df['HOME_SCORE'], errors='coerce')
df['AWAY_SCORE'] = pd.to_numeric(df['AWAY_SCORE'], errors='coerce')




In [ ]:
# Create columns that indicate if there's a change in HOME_SCORE or AWAY_SCORE
df['HOME_SCORE_CHANGE'] = df['HOME_SCORE'].diff().fillna(0) != 0
df['AWAY_SCORE_CHANGE'] = df['AWAY_SCORE'].diff().fillna(0) != 0


In [ ]:
# Count the number of times each score changes
home_score_changes = df['HOME_SCORE_CHANGE'].sum()
away_score_changes = df['AWAY_SCORE_CHANGE'].sum()

print(f"Home score changes: {home_score_changes}")
print(f"Away score changes: {away_score_changes}")


Home score changes: 11091897
Away score changes: 11198320


In [ ]:

home_score_changes = home_score_changes.copy()
home_score_changes.loc[:, 'TIME_LEFT'] = home_score_changes['MINS_LEFT'] * 60 + home_score_changes['SECS_LEFT']
home_score_changes.loc[:, 'TIME_DIFF'] = home_score_changes['TIME_LEFT'].diff().fillna(0)

away_score_changes = away_score_changes.copy()
away_score_changes.loc[:, 'TIME_LEFT'] = away_score_changes['MINS_LEFT'] * 60 + away_score_changes['SECS_LEFT']
away_score_changes.loc[:, 'TIME_DIFF'] = away_score_changes['TIME_LEFT'].diff().fillna(0)



In [ ]:

missing_percentage = df.isnull().mean() * 100


print(missing_percentage.sort_values(ascending=False))


SHOT_CLOCK_RANGE    14.354954
SEASON_1            0.000000 
FGM                 0.000000 
SPD                 0.000000 
DIST                0.000000 
ORBC                0.000000 
DRBC                0.000000 
RBC                 0.000000 
TCHS                0.000000 
SAST                0.000000 
PASS                0.000000 
AST                 0.000000 
FGA                 0.000000 
SEASON_2            0.000000 
FG_PCT_y            0.000000 
DFGM                0.000000 
DFGA                0.000000 
DFG_PCT             0.000000 
SCORE               0.000000 
SCOREMARGIN         0.000000 
EFG_PCT             0.000000 
PERSON1TYPE         0.000000 
PERSON2TYPE         0.000000 
SECS_LEFT           0.000000 
MINS_LEFT           0.000000 
QUARTER             0.000000 
AWAY_TEAM           0.000000 
TEAM_ID             0.000000 
TEAM_NAME           0.000000 
PLAYER_ID           0.000000 
PLAYER_NAME         0.000000 
POSITION_GROUP      0.000000 
POSITION            0.000000 
GAME_DATE 

In [ ]:
# Forward fill for score-related columns within each game
score_columns = ['SCORE', 'SCOREMARGIN', 'HOME_SCORE', 'AWAY_SCORE']
df[score_columns] = df.groupby('GAME_ID')[score_columns].ffill()

# Median imputation for player tracking stats
tracking_stats = ['SPD', 'FG_PCT_y', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'PASS', 'AST',
                  'FGM', 'FGA', 'DFGA', 'DFGM', 'DFG_PCT', 'EFG_PCT',
                  'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE', 'DIST']
# Median imputation for player tracking stats without using inplace
for col in tracking_stats:
    df[col] = df[col].fillna(df[col].median())




In [ ]:
# Replace missing values using the median explicitly with apply and lambda
for col in tracking_stats:
    df[col] = df[col].apply(lambda x: x if pd.notnull(x) else df[col].median())


In [ ]:
df

,LOC_X
0,0.1
1,0.1
2,0.1
3,0.1
4,0.1
...,...
20134689,1.1
20134690,1.1
20134691,1.1
20134692,1.1


In [ ]:
df['SHOT_CLOCK_RANGE'] = df['SHOT_CLOCK_RANGE'].cat.add_categories("Unknown")
df['SHOT_CLOCK_RANGE'].fillna("Unknown", inplace=True)



<ipython-input-10-b89aff22f7cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SHOT_CLOCK_RANGE'] = df['SHOT_CLOCK_RANGE'].cat.add_categories("Unknown")
<ipython-input-10-b89aff22f7cf>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SHOT_CLOCK_RANGE'].fillna("Unknown", inplace

In [ ]:
for col in df.select_dtypes(include=['object']).columns:
    unique_values = df[col].nunique()
    print(f"Unique values in column '{col}': {unique_values}")

Unique values in column 'TEAM_NAME': 30
Unique values in column 'PLAYER_NAME': 458
Unique values in column 'POSITION_GROUP': 3
Unique values in column 'POSITION': 12
Unique values in column 'GAME_DATE': 26
Unique values in column 'HOME_TEAM': 30
Unique values in column 'AWAY_TEAM': 30
Unique values in column 'ACTION_TYPE': 48
Unique values in column 'BASIC_ZONE': 7
Unique values in column 'ZONE_NAME': 6
Unique values in column 'ZONE_ABB': 6
Unique values in column 'ZONE_RANGE': 5
Unique values in column 'SCORE': 5446
Unique values in column 'SCOREMARGIN': 290


In [ ]:

categorical_columns = [
    'TEAM_NAME', 'PLAYER_NAME', 'POSITION_GROUP', 'POSITION',
    'GAME_DATE', 'HOME_TEAM', 'AWAY_TEAM', 'BASIC_ZONE',
    'ZONE_NAME', 'ZONE_ABB', 'ZONE_RANGE'
]
for col in categorical_columns:
    df[col] = df[col].astype('category')


In [ ]:
df.dtypes

,0
SEASON_1,int16
SEASON_2,category
TEAM_ID,int32
TEAM_NAME,category
PLAYER_ID,int32
PLAYER_NAME,category
POSITION_GROUP,category
POSITION,category
GAME_DATE,category
GAME_ID,int32


In [ ]:
text_columns = ['TEAM_NAME', 'PLAYER_NAME', 'POSITION_GROUP', 'POSITION',
                'HOME_TEAM', 'AWAY_TEAM', 'ACTION_TYPE']

for col in text_columns:
    df[col] = df[col].str.strip().str.title()


In [ ]:
continuous_columns = ['SPD', 'FG_PCT_y', 'DFG_PCT', 'EFG_PCT','LOC_X']

scaler = StandardScaler()
df[continuous_columns] = scaler.fit_transform(df[continuous_columns])


In [ ]:
if 'SCORE' in df.columns:
    score_split = df['SCORE'].str.split(' - ', expand=True)
    df['SCORE_HOME'] = pd.to_numeric(score_split[0], errors='coerce')
    df['SCORE_AWAY'] = pd.to_numeric(score_split[1], errors='coerce')
    df.drop(columns=['SCORE'], inplace=True)

In [ ]:

df['SCOREMARGIN'] = pd.to_numeric(df['SCOREMARGIN'].replace('TIE', 0), errors='coerce')
df['SEASON_1'] = df['SEASON_1'].astype('int16')
df['TEAM_ID'] = df['TEAM_ID'].astype('int32')
df['PLAYER_ID'] = df['PLAYER_ID'].astype('int32')
df['GAME_ID'] = df['GAME_ID'].astype('int32')
df['SHOT_DISTANCE'] = df['SHOT_DISTANCE'].astype('float32')
df['SCOREMARGIN'] = df['SCOREMARGIN'].astype('float32')

int_cols = df.select_dtypes(include=['int']).columns
df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')

float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')


In [ ]:
df.to_parquet('/content/drive/MyDrive/optimized_data.parquet', index=False, compression='gzip')
